In [7]:
import os
import cv2
import sys
videoInput = 'C:/Users/Maria/Desktop/GitHub/ComputerVision/Curs6-Python-Yolo pe video/drop.avi'

if not os.path.isfile(videoInput):
    print("Input video file ", videoInput, " doesn't exist")
    sys.exit(1)
cap = cv2.VideoCapture(videoInput)
fisierIesire = videoInput[:-4] + '_yolo.avi'
frameTotal = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print('Numarul de frame-uri este ', frameTotal)
# Salvam fisierul video
videoOut = cv2.VideoWriter(fisierIesire, cv2.VideoWriter_fourcc('M','J','P','G'),
    30, (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

Numarul de frame-uri este  182.0


In [20]:

pas = 0
while cv2.waitKey(1) < 0:
    # citim frame de la video
    maiSuntFrameuri, frame = cap.read()
    if(pas % 10 == 0):# Afisam mesaj din 10 in 10 frame-uri
        print('Am procesat {} din {} de frame-uri.'.format(pas, frameTotal))

    # Daca s-a terminat fisierul video
    if not maiSuntFrameuri:
        print("Fisierul de iesire este gata: ", fisierIesire)
        cv2.waitKey(3000)
        break
    (hFrame, wFrame) = frame.shape[:2]

    ######### Luam partea de predictie si antrenare de mai sus #########
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (inWidth, inHeight),
                                                    swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    #adunam in liste
    dreptunghiuri = []
    increderi = []
    clase = []

    # pentru fiecare iesire
    for output in layerOutputs:
        # pentru fiecare obiect detectat
        for detectare in output:
            '''
            Extragem clasa si probabilitatea (increderea) obiectului curent.
            8
            Acesta conteaza doar daca are probabilitatea > pragIncredere.
            Primele 4 elemente reprezinta centru_x, centru_y, lungime,
            inaltime. Al 5-lea element este increderea ca dreptunghiul
            cuprinde un obiect.
            '''
            scoruri = detectare[5:]
            clasa = np.argmax(scoruri)
            incredere = scoruri[clasa]
            if incredere > pragIncredere:
                # scalam dreptunghiul relativ la dimensiunea initiala a pozei
                dreptunghi = detectare[0:4] * np.array([wFrame, hFrame, wFrame, hFrame])
                (centruX, centruY, lungime, inaltime) = dreptunghi.astype("int")
                x = int(centruX - (lungime / 2))
                y = int(centruY - (inaltime / 2))
                dreptunghiuri.append([x, y, int(lungime), int(inaltime)])
                increderi.append(float(incredere))
                clase.append(clasa)

    idxs = cv2.dnn.NMSBoxes(dreptunghiuri, increderi, pragIncredere, pragSuprimare)
    # daca avem cel putin o detectie
    if len(idxs) > 0:
        for i in idxs.flatten():
            # dreptunghiurile
            (x, y) = (dreptunghiuri[i][0], dreptunghiuri[i][1])
            (w, h) = (dreptunghiuri[i][2], dreptunghiuri[i][3])
            # desenam dreptunghiurile si etichetele
            culoare = [int(c) for c in culori[clase[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), culoare, 2)
            text = "{}: {:.4f}".format(etichete[clase[i]], increderi[i])
            cv2.putText(frame, text, (x, y - 5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, culoare, 2)
    ######### Pana aici am luat antrenarea si predictia#########
    # Salvam frame-ul cu adnotari
    videoOut.write(frame.astype(np.uint8))
    pas += 1
videoOut.release()

C:/Users/Maria/Desktop/GitHub/ComputerVision/Curs6-Python-Yolo pe video/COCOyolov3.cfg
C:/Users/Maria/Desktop/GitHub/ComputerVision/Curs6-Python-Yolo pe video/COCOyolov3.weights
Incarcam modelul antrenat anterior (80 de clase)


error: OpenCV(3.4.5) C:\projects\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:207: error: (-212:Parsing error) Failed to parse NetParameter file: C:/Users/Maria/Desktop/GitHub/ComputerVision/Curs6-Python-Yolo pe video/COCOyolov3.cfg in function 'cv::dnn::experimental_dnn_34_v11::readNetFromDarknet'
